# Coleta e preparação de dados

In [18]:
!pip install pandas==2.0.0

In [19]:
import pandas as pd

In [20]:
df = pd.read_csv('https://github.com/lukaswyllis/formacao-avancando-em-data-science-com-python/raw/refs/heads/master/dados-curso4/relatorio_vendas.csv')

In [21]:
df.head()

,data_pedido,modo_envio,nome_cliente,segmento_cliente,cidade,estado,regiao,departamento,tipo_produto,vendas,quantidade,lucro
0,2018-11-09,Econômica,Thiago Silveira,B2C,Ribeirão Preto,São Paulo,Sudeste,Materiais de construção,encanamentos,890.66,2,142.51
1,2018-11-09,Econômica,Thiago Silveira,B2C,Ribeirão Preto,São Paulo,Sudeste,Materiais de construção,ferramentas,2488.60,3,746.58
2,2018-06-13,Econômica,Giovanna Lima,B2B,Rio de Janeiro,Rio de Janeiro,Sudeste,Jardinagem e paisagismo,sementes,49.71,2,23.36
3,2017-10-12,Entrega padrão,Ana Júlia da Cruz,B2C,Foz do Iguaçu,Paraná,Sul,Materiais de construção,materiais de revestimento,3255.76,5,-1302.31
4,2017-10-12,Entrega padrão,Ana Júlia da Cruz,B2C,Foz do Iguaçu,Paraná,Sul,Jardinagem e paisagismo,vasos,76.05,2,8.56


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8940 entries, 0 to 8939
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   data_pedido       8940 non-null   object 
 1   modo_envio        8940 non-null   object 
 2   nome_cliente      8940 non-null   object 
 3   segmento_cliente  8940 non-null   object 
 4   cidade            8940 non-null   object 
 5   estado            8940 non-null   object 
 6   regiao            8940 non-null   object 
 7   departamento      8940 non-null   object 
 8   tipo_produto      8940 non-null   object 
 9   vendas            8940 non-null   float64
 10  quantidade        8940 non-null   int64  
 11  lucro             8940 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 838.3+ KB


# Missão 1: Time de vendas

Precisamos construir uma visualização para indicar os **10 maiores clientes por total de venda**.

Com isso, a equipe de vendas pode pensar em estratégias para manter esses clientes comprando em nossa loja.

In [23]:
df_cliente_vendas = df.groupby(['nome_cliente'])['vendas'].sum().nlargest(10).copy()
df_cliente_vendas = df_cliente_vendas.reset_index()
df_cliente_vendas.columns = ['Clientes', 'Vendas']
df_cliente_vendas['Ranking'] = df_cliente_vendas.index + 1
df_cliente_vendas.set_index(df_cliente_vendas['Ranking'], inplace=True)
df_cliente_vendas.drop('Ranking', axis=1, inplace=True)
df_cliente_vendas

,Clientes,Vendas
Ranking,,
1,Maria Luiza Almeida,64777.54
2,Ana Julia Pinto,51398.95
3,Ryan Farias,48178.94
4,Heitor da Mata,47610.85
5,Maria Clara Gonçalves,46946.42
6,Raquel Freitas,44826.26
7,Davi Ramos,43769.21
8,Amanda Melo,42354.27
9,Alexia Ribeiro,41238.84


In [24]:
s = df_cliente_vendas.style
s.format({'Vendas': 'R$ {:,.2f}'})
s

,Clientes,Vendas
Ranking,,
1,Maria Luiza Almeida,"R$ 64,777.54"
2,Ana Julia Pinto,"R$ 51,398.95"
3,Ryan Farias,"R$ 48,178.94"
4,Heitor da Mata,"R$ 47,610.85"
5,Maria Clara Gonçalves,"R$ 46,946.42"
6,Raquel Freitas,"R$ 44,826.26"
7,Davi Ramos,"R$ 43,769.21"
8,Amanda Melo,"R$ 42,354.27"
9,Alexia Ribeiro,"R$ 41,238.84"


## Desafio 1
O primeiro desafio é construir uma visualização que permita identificar quais países mais solicitaram produtos por pedidos para que assim possa ser feito um estudo quanto a distribuição e logística de produtos.

In [25]:
df_loja_livro_filmes = pd.read_csv('https://raw.githubusercontent.com/lukaswyllis/formacao-avancando-em-data-science-com-python/refs/heads/master/dados-curso4/loja_livro_filmes.csv')

In [26]:
df_paises = df_loja_livro_filmes.groupby('pais')['unidades'].sum().nlargest(10).copy()
df_paises = df_paises.reset_index()
df_paises.columns = ['País', 'Unidades']
df_paises['Rank'] = df_paises.index + 1
df_paises.set_index('Rank', inplace=True)
s_paises = df_paises.style
s_paises

,País,Unidades
Rank,,
1,Argentina,5591
2,Equador,5566
3,Paraguai,5564
4,Peru,5550
5,Venezuela,5529
6,Colômbia,5504
7,Brasil,5473
8,Uruguai,5377
9,Chile,5304


# Missão 2: Time comercial

Precisamos construir uma visualização que permita associar o **total ganho em vendas e o total lucrado por tipo de produto**.

Isso vai permitir que o time comercial possa focar em estratégias para alavancar as rendas que ganham com cada tipo de produto vendido na loja.

In [27]:
df_venda_lucro = df.groupby(['tipo_produto'])[['vendas', 'lucro']].sum()
df_venda_lucro.index.name = 'Tipo Produto'
df_venda_lucro

,vendas,lucro
Tipo Produto,,
decoração de jardim,82680.87,19880.86
encanamentos,373224.39,-11243.39
equipamentos de limpeza,542304.55,17448.30
ferramentas,995159.43,82042.91
ferramentas automotivas,502109.33,126660.54
ferramentas de jardinagem,648880.47,106408.80
fertilizantes,53144.55,22509.86
iluminação,275229.82,40531.09
materiais de paisagismo,150552.80,-3823.67


In [28]:
estilo_produto = df_venda_lucro.style
estilo_produto

,vendas,lucro
Tipo Produto,,
decoração de jardim,82680.870000,19880.860000
encanamentos,373224.390000,-11243.390000
equipamentos de limpeza,542304.550000,17448.300000
ferramentas,995159.430000,82042.910000
ferramentas automotivas,502109.330000,126660.540000
ferramentas de jardinagem,648880.470000,106408.800000
fertilizantes,53144.550000,22509.860000
iluminação,275229.820000,40531.090000
materiais de paisagismo,150552.800000,-3823.670000


In [33]:
estilo_produto.format('R$ {:,.2f}')\
              .highlight_max(color='darkgreen')\
              .highlight_min(color='#F16165')

,vendas,lucro
Tipo Produto,,
decoração de jardim,"R$ 82,680.87","R$ 19,880.86"
encanamentos,"R$ 373,224.39","R$ -11,243.39"
equipamentos de limpeza,"R$ 542,304.55","R$ 17,448.30"
ferramentas,"R$ 995,159.43","R$ 82,042.91"
ferramentas automotivas,"R$ 502,109.33","R$ 126,660.54"
ferramentas de jardinagem,"R$ 648,880.47","R$ 106,408.80"
fertilizantes,"R$ 53,144.55","R$ 22,509.86"
iluminação,"R$ 275,229.82","R$ 40,531.09"
materiais de paisagismo,"R$ 150,552.80","R$ -3,823.67"


In [34]:
estilo_produto.format('R$ {:,.2f}').background_gradient(cmap='Greens')
estilo_produto

,vendas,lucro
Tipo Produto,,
decoração de jardim,"R$ 82,680.87","R$ 19,880.86"
encanamentos,"R$ 373,224.39","R$ -11,243.39"
equipamentos de limpeza,"R$ 542,304.55","R$ 17,448.30"
ferramentas,"R$ 995,159.43","R$ 82,042.91"
ferramentas automotivas,"R$ 502,109.33","R$ 126,660.54"
ferramentas de jardinagem,"R$ 648,880.47","R$ 106,408.80"
fertilizantes,"R$ 53,144.55","R$ 22,509.86"
iluminação,"R$ 275,229.82","R$ 40,531.09"
materiais de paisagismo,"R$ 150,552.80","R$ -3,823.67"


In [35]:
cabecalho = {
    'selector':'th',
    'props':'font-weigth: bold; font-family: Arial; text-align: center; text-transform: capitalize;'
}

In [36]:
estilo_produto.set_table_styles([cabecalho], overwrite=False)

,vendas,lucro
Tipo Produto,,
decoração de jardim,"R$ 82,680.87","R$ 19,880.86"
encanamentos,"R$ 373,224.39","R$ -11,243.39"
equipamentos de limpeza,"R$ 542,304.55","R$ 17,448.30"
ferramentas,"R$ 995,159.43","R$ 82,042.91"
ferramentas automotivas,"R$ 502,109.33","R$ 126,660.54"
ferramentas de jardinagem,"R$ 648,880.47","R$ 106,408.80"
fertilizantes,"R$ 53,144.55","R$ 22,509.86"
iluminação,"R$ 275,229.82","R$ 40,531.09"
materiais de paisagismo,"R$ 150,552.80","R$ -3,823.67"


## **Desafio 2: hora da prática**

Aprendemos a construir uma visualização capaz de destacar elementos através de algumas funções built-in. E, nesse desafio, podemos aplicar algumas dessas funções.

O objetivo desse desafio é construir uma visualização que permita **identificar quais categorias de produto geraram mais e menos custos para a loja**.

In [58]:
df_categoria = df_loja_livro_filmes.groupby('categoria')['custo_produto'].sum()
df_categoria = df_categoria.reset_index()
df_categoria.columns = ['Categoria', 'Custo dos Produtos']
estilo_categoria = df_categoria.style

estilo_categoria.format('R$ {:,.2f}')
# \
#               .highlight_max(color='darkgreen')\
#               .highlight_min(color='#F16165')

ValueError: Unknown format code 'f' for object of type 'str'

# Missão 3: Time de logística

Precisamos construir uma visualização que permita entender a **distribuição de pedidos por região do Brasil**.

Isso vai permitir que o time logística entenda onde é mais interessante focar material e pessoas.

# Missão 4: Time de logística

Novamente o time de logística deseja repensar sua estrutura e organização de fornecimento de produtos e para isso, foi pensada em uma visualização que permita identificar padrão de **quantidade de produtos solicitados por mês por departamento**.

# Missão 5: Relatório de performance

O relatório de performance é um documento muito importante para a empresa. Nele, todos que trabalham na loja podem entender como anda a empresa. Por conta disso, tabelas bem construídas são essenciais para mostrar os dados da melhor forma.

Portanto, precisamos construir uma visualização que permita entender a **relação entre tipos de clientes e modo de envio de produtos de acordo com as vendas**. Com isso, será possível destacar no relatório qual o modo de envio e quais tipos de modo de envio mais geraram ganho em vendas.